In [53]:
pip install -q google-adk[a2a]

Note: you may need to restart the kernel to use updated packages.


In [54]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )


✅ Setup and authentication complete.


In [55]:
import json
import requests
import subprocess
import time
import uuid

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types


# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [56]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [57]:
import pandas as pd

df_meals = pd.read_csv("/kaggle/input/meals-data/meals.csv")
df_meals.head()


,meal_id,name,cuisine,meal_type,diet_type,calories,protein_g,carbs_g,fat_g,portion_label,serving_multiplier,is_vegetarian,is_gluten_free,is_keto_friendly,cost_per_serving_usd,prep_time_min
0,1,Keto Chicken Bowl — Single Portion,Mediterranean,dinner,keto,594,32,20,49,Single Portion,1.0,False,True,True,5.91,16
1,2,Keto Chicken Bowl — Large Serving,Mediterranean,dinner,keto,1011,72,7,61,Large Serving,1.5,False,True,True,5.59,42
2,3,Keto Chicken Bowl — Single Portion,Asian,breakfast,keto,376,57,2,63,Single Portion,1.0,False,True,True,11.73,14
3,4,Keto Chicken Bowl — Single Portion,Mediterranean,lunch,keto,303,22,19,37,Single Portion,1.0,False,True,True,9.23,22
4,5,Keto Chicken Bowl — Single Portion,Middle Eastern,lunch,keto,755,48,17,48,Single Portion,1.0,False,True,True,8.28,22


In [58]:
# Hard coded user profile 

user_profile = {
    "diet_type": "high_protein",        # or keto, gluten_free, etc.
    "daily_calorie_target": 1760,       # deficit included
    "daily_protein_target": 167,        # grams
    "max_cost_per_day": 12.0,           # USD
    "max_prep_time_per_day": 45,        # minutes total
    "avoid_ingredients": ["beef"],    # allergies / dislikes
    "pref_meal_types": ["lunch", "dinner"],  # if they care
}

In [59]:
# list of meal_ids or rows from df_meals (starting point)
candidate_plan = {
    "day": "Monday",
    "meals": [
        {"slot": "breakfast", "meal_id": 3},
        {"slot": "lunch",     "meal_id": 47},
        {"slot": "dinner",    "meal_id": 82},
    ]
}


Later the Coordinator Agent will generate these candidates and send them to other agents to score.

In [60]:
def sample_daily_plan(df, user_profile):
    # filter by diet
    df_filt = df[df["diet_type"] == user_profile["diet_type"]].copy()

    # very naive: just pick 3 random meals
    chosen = df_filt.sample(3, replace=False).reset_index(drop=True)

    slots = ["breakfast", "lunch", "dinner"]
    meals = []
    for slot, (_, row) in zip(slots, chosen.iterrows()):
        meals.append({
            "slot": slot,
            "meal_id": int(row["meal_id"])
        })

    return {
        "day": "Monday",
        "meals": meals
    }

candidate_plan = sample_daily_plan(df_meals, user_profile)
candidate_plan


{'day': 'Monday',
 'meals': [{'slot': 'breakfast', 'meal_id': 384},
  {'slot': 'lunch', 'meal_id': 398},
  {'slot': 'dinner', 'meal_id': 441}]}

In [61]:
def get_meal_options(
    diet_type: str,
    max_calories: int = 800,
    max_cost_per_serving_usd: float = 10.0,
    meal_type: str = "",     # changed to simple str default
    top_k: int = 5
) -> dict:
    """
    Get a list of meals that fit basic user constraints.

    Args:
        diet_type: e.g. "keto", "high_protein", "bulking"
        max_calories: max calories per meal
        max_cost_per_serving_usd: budget per meal
        meal_type: optional filter like "breakfast", "lunch", "dinner", or "" for no filter
        top_k: how many suggestions to return

    Returns:
        dict with structure:
          - status: "success" or "error"
          - count: number of meals found
          - meals: list of meal dicts (if success)
          - error_message: string (if error)
    """
    df = df_meals.copy()

    df = df[df["diet_type"] == diet_type]
    df = df[df["calories"] <= max_calories]
    df = df[df["cost_per_serving_usd"] <= max_cost_per_serving_usd]

    if meal_type:
        df = df[df["meal_type"] == meal_type]

    if df.empty:
        return {
            "status": "error",
            "count": 0,
            "meals": [],
            "error_message": (
                f"No meals found for diet_type={diet_type}, "
                f"max_calories={max_calories}, "
                f"max_cost_per_serving_usd={max_cost_per_serving_usd}, "
                f"meal_type={meal_type}"
            )
        }

    df = df.sample(min(top_k, len(df)), random_state=0)

    meals_list = []
    for _, row in df.iterrows():
        meals_list.append({
            "meal_id": int(row["meal_id"]),
            "name": str(row["name"]),
            "meal_type": str(row["meal_type"]),
            "diet_type": str(row["diet_type"]),
            "calories": int(row["calories"]),
            "protein_g": int(row["protein_g"]),
            "cost_per_serving_usd": float(row["cost_per_serving_usd"]),
            "prep_time_min": int(row["prep_time_min"])
        })

    return {
        "status": "success",
        "count": len(meals_list),
        "meals": meals_list
    }


In [62]:
meal_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="meal_catalog_agent",
    description="Agent that suggests meals from a structured catalog based on diet, calories, and budget.",
    instruction="""
    You are a meal planning assistant.
    When the user asks for meal ideas, always call the get_meal_options tool
    with sensible parameters based on their goals (diet_type, calories, cost, and meal_type).
    You respond with clear, concise suggestions.
    If nothing matches, suggest relaxing constraints.
    """,
    tools=[get_meal_options],
)

print("✅ Meal Catalog Agent created successfully!")


✅ Meal Catalog Agent created successfully!


In [63]:
from google.adk.a2a.utils.agent_to_a2a import to_a2a

meal_prep_a2a_app = to_a2a(meal_catalog_agent, port=8001)

print("✅ A2A app created for meal_catalog_agent on port 8001")


✅ A2A app created for meal_catalog_agent on port 8001


In [64]:
import os

meal_catalog_agent_code = '''
import os
import pandas as pd

from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

# Retry config
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# Load the meal catalog CSV
df_meals = pd.read_csv("/kaggle/input/meals-data/meals.csv")

def get_meal_options(
    diet_type: str,
    max_calories: int = 800,
    max_cost_per_serving_usd: float = 10.0,
    meal_type: str = "",        # changed from Optional[str] to str with default ""
    top_k: int = 5,
) -> dict:
    """
    Look up meals that match simple constraints and return a JSON‐safe dict.
    """
    df = df_meals.copy()

    df = df[df["diet_type"] == diet_type]
    df = df[df["calories"] <= max_calories]
    df = df[df["cost_per_serving_usd"] <= max_cost_per_serving_usd]

    if meal_type:
        df = df[df["meal_type"] == meal_type]

    if df.empty:
        return {
            "status": "error",
            "count": 0,
            "meals": [],
            "error_message": (
                f"No meals found for diet_type={diet_type}, "
                f"max_calories={max_calories}, "
                f"max_cost_per_serving_usd={max_cost_per_serving_usd}, "
                f"meal_type={meal_type}"
            )
        }

    df = df.sample(min(top_k, len(df)), random_state=0)

    meals_list = []
    for _, row in df.iterrows():
        meals_list.append({
            "meal_id": int(row["meal_id"]),
            "name": str(row["name"]),
            "meal_type": str(row["meal_type"]),
            "diet_type": str(row["diet_type"]),
            "calories": int(row["calories"]),
            "protein_g": int(row["protein_g"]),
            "cost_per_serving_usd": float(row["cost_per_serving_usd"]),
            "prep_time_min": int(row["prep_time_min"]),
        })

    return {
        "status": "success",
        "count": len(meals_list),
        "meals": meals_list
    }

meal_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="meal_catalog_agent",
    description="External meal catalog agent that suggests meals based on diet, calories, and budget.",
    instruction="""
You are a meal planning assistant for an external meal catalog.
When the user asks for meal ideas, use the get_meal_options tool
with sensible parameters based on their goals (diet_type, calories, cost, and meal_type).
Respond with clear, concise suggestions.
If nothing matches, suggest relaxing constraints.
""",
    tools=[get_meal_options],
)

# Expose as A2A app
app = to_a2a(meal_catalog_agent, port=8001)

'''

# Write the meal catalog agent to a temporary file
with open("/tmp/meal_catalog_server.py", "w") as f:
    f.write(meal_catalog_agent_code)

print("📝 Meal Catalog agent code saved to /tmp/meal_catalog_server.py")


📝 Meal Catalog agent code saved to /tmp/meal_catalog_server.py


In [65]:
server_process = subprocess.Popen(
    [
        "uvicorn",
        "meal_catalog_server:app",  # module:app
        "--host",
        "localhost",
        "--port",
        "8001",
    ],
    cwd="/tmp",
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},
)

print("🚀 Starting Meal Catalog Agent server...")
print("   Waiting for server to be ready...")

max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8001/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ Meal Catalog Agent server is running!")
            print(f"   Server URL: http://localhost:8001")
            print(f"   Agent card: http://localhost:8001/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")

globals()["meal_catalog_server_process"] = server_process

🚀 Starting Meal Catalog Agent server...
   Waiting for server to be ready...

✅ Meal Catalog Agent server is running!
   Server URL: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json


In [66]:
# Fetch the agent card from the running server
try:
    response = requests.get(
        "http://localhost:8001/.well-known/agent-card.json", timeout=5
    )

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 Meal Prep Agent Card:")
        print(json.dumps(agent_card, indent=2))

        print("\n✨ Key Information:")
        print(f"   Name: {agent_card.get('name')}")
        print(f"   Description: {agent_card.get('description')}")
        print(f"   URL: {agent_card.get('url')}")
        print(f"   Skills: {len(agent_card.get('skills', []))} capabilities exposed")
    else:
        print(f"❌ Failed to fetch agent card: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")
    print("   Make sure the Meal Prep Agent server is running on port 8001.")

📋 Meal Prep Agent Card:
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "External meal catalog agent that suggests meals based on diet, calories, and budget.",
  "name": "meal_catalog_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "External meal catalog agent that suggests meals based on diet, calories, and budget. \nI am a meal planning assistant for an external meal catalog.\nWhen the user asks for meal ideas, use the get_meal_options tool\nwith sensible parameters based on their goals (diet_type, calories, cost, and meal_type).\nRespond with clear, concise suggestions.\nIf nothing matches, suggest relaxing constraints.\n",
      "id": "meal_catalog_agent",
      "name": "model",
      "tags": [
        "llm"
      ]
    },
    {
      "description": "Look up meals that match simple constraints and return a JSON\u2010safe dict.",
 

In [67]:
# Create a RemoteA2aAgent that connects to our Meal Prep Agent
# This acts as a client-side proxy so other agents can call its skills
remote_meal_prep_agent = RemoteA2aAgent(
    name="meal_prep_agent",
    description="Remote Meal Prep Agent that handles meal retrieval, filtering, and meal plan creation.",
    # Point to the agent-card.json (A2A metadata)
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote Meal Prep Agent proxy created!")
print(f"   Connected to: http://localhost:8001")
print(f"   Agent card: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")
print("   Other agents can now use the Meal Prep Agent like a local sub-agent!")


✅ Remote Meal Prep Agent proxy created!
   Connected to: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json
   Other agents can now use the Meal Prep Agent like a local sub-agent!


In [68]:
nutrition_coach_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="nutrition_coach_agent",
    description="A high-level nutrition assistant that generates personalized meal plans using the Meal Prep Agent.",
    instruction="""
    You are a friendly, smart Nutrition Coach.

    Your job:
    1. Understand the user's dietary goals (calories, macros, diet type, budget, prep time).
    2. ALWAYS call the meal_prep_agent sub-agent to fetch meals, filter meals, 
       or generate daily/weekly meal plans.
    3. Do NOT invent meals on your own — always ask the meal_prep_agent for real data.
    4. After receiving meal options or a plan from the sub-agent, summarize it clearly.
    5. Keep your tone helpful and encouraging.

    Rules:
    - Never hallucinate meals or macros.
    - Always defer to meal_prep_agent for actual meal information.
    - You are the coordinator, not the meal generator.
    """,
    sub_agents=[remote_meal_prep_agent],  # Add the remote Meal Prep Agent as a sub-agent!
)

print("✅ Nutrition Coach Agent created!")
print("   Model: gemini-2.5-flash-lite")
print("   Sub-agents: 1 (remote Meal Prep Agent)")
print("   Ready to generate personalized meal plans!")

✅ Nutrition Coach Agent created!
   Model: gemini-2.5-flash-lite
   Sub-agents: 1 (remote Meal Prep Agent)
   Ready to generate personalized meal plans!


In [70]:
async def test_meal_prep_a2a(user_query: str) -> None:
    """
    Test the A2A-style communication between the Nutrition Coach Agent and Meal Prep Agent.

    This function:
    1. Creates a new session for this conversation
    2. Sends the query to the Nutrition Coach Agent
    3. The Nutrition Coach Agent delegates to the Meal Prep Agent via sub-agent call
    4. Displays the final response

    Args:
        user_query: The user's nutrition / meal-planning request
    """
    # Setup session management (required by ADK)
    session_service = InMemorySessionService()

    # Session identifiers
    app_name = "meal_prep_app"
    user_id = "demo_user"
    session_id = f"meal_session_{uuid.uuid4().hex[:8]}"

    # Create session BEFORE running agent
    await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
    )

    # Create the runner for the Nutrition Coach Agent
    runner = Runner(
        agent=nutrition_coach_agent,
        app_name=app_name,
        session_service=session_service,
    )

    # Create the user message
    test_content = types.Content(parts=[types.Part(text=user_query)])

    # Display query
    print(f"\n👤 User: {user_query}")
    print(f"\n🥗 Nutrition Coach response:")
    print("-" * 60)

    # Run the agent asynchronously (handles streaming + sub-agent calls)
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=test_content,
    ):
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if getattr(part, "text", None):
                    print(part.text)

    print("-" * 60)


# Run the test
print("🧪 Testing Nutrition Coach ↔ Meal Prep Agent communication...\n")
await test_meal_prep_a2a(
    "I need around 800 calorie, high-protein and indian meal, low-carb meal plan for 1 day. Keep prep time under 35 minutes per meal."
)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False


🧪 Testing Nutrition Coach ↔ Meal Prep Agent communication...


👤 User: I need around 800 calorie, high-protein and indian meal, low-carb meal plan for 1 day. Keep prep time under 35 minutes per meal.

🥗 Nutrition Coach response:
------------------------------------------------------------


INFO:google_adk.google.adk.models.google_llm:Response received from the model.


Sorry, I couldn't find any meals that fit all your criteria. You might have more luck if you relax some constraints, such as the meal type.
------------------------------------------------------------
